In [1]:
import json
from glob import glob
from multiprocessing import Pool
from pathlib import Path
import os
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from stexls.util import file_watcher
from pickle import dumps, loads, dump, load

from stexls.util.latex import parser

from stexls import *

%cd ~/MathHub

/home/marian/MathHub


In [2]:
class Workspace:
    def __init__(self, root: Path, limit: int = -1):
        self.limit = limit
        self.root = root
        self.watcher = file_watcher.WorkspaceWatcher(root, filter='*.tex')
        self.compiled_objects: Dict[Path, List[StexObject]] = {}
        self.module_index: Dict[Path, Dict[str, StexObject]] = {}
        self.build_orders: Dict[StexObject, List[StexObject]] = {}
        self.changes = None
    
    def update(self, use_multiprocessing: bool = True):
        self.changes = self.watcher.update()
        changelist = list(self.changes.created | self.changes.modified)[:self.limit]
        print(f'{len(changelist)} changes. {len(self.changes.deleted)} files deleted.')
        for changed in (self.changes.deleted | self.changes.created | self.changes.modified):
            if changed in self.compiled_objects:
                del self.compiled_objects[changed]
        with Pool() as pool:
            mapfn = pool.map if use_multiprocessing else map
            parsed_files = list(mapfn(parse, changelist))
            parsed_files = {
                file: parsed
                for file, parsed
                in zip(changelist, parsed_files)
                if parsed
            }
            self.compiled_objects.update({
                file: objects
                for file, objects
                in zip(
                    parsed_files.keys(),
                    mapfn(
                        Workspace._compile,
                        parsed_files.values())
                )
            })
        self.module_index = {}
        for path, objects in self.compiled_objects.items():
            for object in objects:
                if object.module:
                    self.module_index.setdefault(path.absolute(), dict())[object.module] = object
        self.build_orders = {
            object: make_build_order(object, self.module_index)
            for objects in self.compiled_objects.values()
            for object in objects
        }

    @staticmethod
    def _compile(*args, **kwargs):
        return list(StexObject.compile(*args, **kwargs))

In [3]:
ws = Workspace('.', limit=-1)

In [4]:
ws.update()

8008 changes. 0 files deleted.
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
Undefined module: "symtax-base" not defined in "/home/marian/MathHub/MiKoMH/GenCS/source/logic/en/syntax-base.tex"
Undefined module: "symtax-base" not defined in "/home/marian/MathHub/MiKoMH/GenCS/source/logic/en/syntax-base.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File

MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/GenCS/source/legal/en/gdpr.tex:4:1: File not indexed:"/home/marian/MathHub/KwarcMH/SiSsi/source/winograd/cds/money.tex"
MiKoMH/GenCS/source/legal/en/gdpr.tex:5:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/siunits-prefixed.tex"
Undefined module: "symtax-base" not defined in "/home/marian/MathHub/MiKoMH/GenCS/source/logic/en/syntax-base.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseM

MiKoMH/talks/source/kwarc/en/projects.tex:5:1: File not indexed:"/home/marian/MathHub/MiKoMH/talks/source/sTeX/en/sTeX-approach.tex"
MiKoMH/talks/source/sTeX/en/stexide.tex:2:1: File not indexed:"/home/marian/MathHub/MiKoMH/talks/source/sTeX/en/sTeX-approach.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/talks/source/arxmliv/en/arxmliv.tex:6:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/siunits-prefixed.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
Undefined module: "symtax-base" not defined in "/home/marian/MathHub/MiKoMH/GenCS/source/logic/en/syntax-base.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/h

MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
Undefined module: "physml-observables" not defined in "/home/marian/MathHub/MiKoMH/talks/source/physml/en/nutshell.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/talks/source/planetary/en/realizing.tex:8:3: File not indexed:"/home/marian/MathHub/MiKoMH/talks/source/sTeX/en/sTeX-approach.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not in

MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/trs/en/confluent.tex:2:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/abstract-reduction-system.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-ca

...

MiKoMH/CompLog/source/trs/en/intro.tex:2:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/abstract-reduction-system.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/trs/en/intro.tex:2:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/abstract-reduction-system.tex"
MiKoMH/CompLog/source/trs/en/intro.tex:2:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/abstract-reduction-system.tex"
MiKoMH/TDM/source/doccomp/en/yacc-ex.tex:2:1: File not indexed:"/home/marian/MathHub/MiKoMH/TDM/source/doccomp/en/yacc.tex"
Undefined module: "sqlite-parasubts" not defined in "/home/marian/MathHub/MiKoMH/IWGS/source/databases/en/sqlite-parasubst.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexe

MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
Undefined module: "python-nutshell" not d

MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
smglom/chevahir/source/maximaltorus.tex:2:1: File not indexed:"/home/marian/MathHub/smglom/algebra/source/algebraictorus.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/GenCS/source/memory/en/units-information.tex:5:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/siunits-prefixed.tex"
MiKoMH/GenCS/source/memory/en/units-information.

MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/SecDep/source/course/fragments/acknowledgements.tex:18:5: File not indexed:"/home/marian/MathHub/MiKoMH/talks/source/sTeX/en/sTeX-approach.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
MiKoMH/CompLog/source/trs/en/intro.tex:2:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/abstract-reduction-system.tex"
MiKoMH/CompLog/source/trs/en/intro.tex:2:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/abstract-reduction-system.tex"
MiKoMH/CompLog/source/stlc/en/lambd

MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/talks/source/arxmliv/en/arxmliv.tex:6:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/siunits-prefixed.tex"
MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
MiKoMH/GenCS/source/memory/en/units-information.tex:5:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/siunits-prefixed.tex"
MiKoMH/GenCS/source/memory/en/units-information.tex:64:7: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/siunits-prefixed.tex"
Undefined module: "symtax-base" not defined in "/home/marian/MathHub/MiKoMH/GenCS/source/logic/en/syntax-base.tex"
Undefined module: "symtax-base" not defined in "/home/marian/MathHub/MiKoMH/GenCS/source/logic/en/syntax-base.tex"
Undefined module: "symtax-base" not defined in "/home/marian/Math

In [5]:
list(sorted((len(b), a.path) for a, b in ws.build_orders.items()))[::-1]

[(209,
  PosixPath('MiKoMH/talks/source/planetary/fragments/demo-presstruct.tex')),
 (200, PosixPath('MiKoMH/talks/source/mathhub/en/tgview.tex')),
 (199, PosixPath('MiKoMH/talks/source/planetary/en/reengineering.tex')),
 (199, PosixPath('MiKoMH/talks/source/mathhub/en/nutshell2.tex')),
 (199, PosixPath('MiKoMH/talks/source/mathhub/en/architecture.tex')),
 (196, PosixPath('MiKoMH/talks/source/planetary/en/mscview.tex')),
 (196, PosixPath('MiKoMH/talks/source/kwarc/tikz/projectplan.tex')),
 (196, PosixPath('MiKoMH/talks/source/kwarc/en/projectplan.tex')),
 (196, PosixPath('MiKoMH/talks/source/kwarc/en/phds.tex')),
 (196, PosixPath('MiKoMH/talks/source/kwarc/en/personell.tex')),
 (196, PosixPath('MiKoMH/talks/source/kwarc/en/mscs.tex')),
 (196, PosixPath('MiKoMH/talks/source/kwarc/de/projectplan.tex')),
 (195, PosixPath('MiKoMH/talks/source/kwarc/en/projects.tex')),
 (193, PosixPath('MiKoMH/talks/source/planetary/en/planetmath-nutshell.tex')),
 (193, PosixPath('MiKoMH/IWGS/source/digdocs